# Plot results for original and new bundle adjustment

In [1]:
import xdem
import geoutils as gu
import os,glob
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites'
site_names = ['MCS'] #['Banner', 'MCS', 'JacksonPeak']

for site_name in site_names:
    # Load reference DEM
    if site_name=='JacksonPeak':
        refdem_fn = os.path.join(data_dir, site_name, 'refdem', 'USGS_LPC_ID_FEMAHQ_2018_D18_merged_filtered.tif')
    else:
        refdem_fn = glob.glob(os.path.join(data_dir, site_name, 'refdem', f'{site_name}_REFDEM_WGS84.tif'))[0]
    refdem = xdem.DEM(refdem_fn)
    # Load DEM dates
    dates = ['20241003'] #sorted(glob.glob(os.path.join(data_dir, site_name, '20*')))
    # Iterate over dates
    for date in dates:
        # Load DEMs
        old_dem_fn = glob.glob(os.path.join(data_dir, site_name, date, f"original_*DEM.tif"))[0]
        old_dem = xdem.DEM(old_dem_fn)
        dem1_fn = os.path.join(data_dir, site_name, date, f"original+DEM_MCS_20241003_DEM.tif")
        dem1 = xdem.DEM(dem1_fn).reproject(old_dem)
        dem2_fn = os.path.join(data_dir, site_name, date, f"original+DEMuncertainty1m_MCS_20241003_DEM.tif")
        dem2 = xdem.DEM(dem2_fn).reproject(old_dem)
        dem3_fn = os.path.join(data_dir, site_name, date, f"original+DEMuncertainty0.1m_MCS_20241003_DEM.tif")
        dem3 = xdem.DEM(dem3_fn).reproject(old_dem)
        dem4_fn = os.path.join(data_dir, site_name, date, f"run-run-trans_source-DEM.tif")
        dem4 = xdem.DEM(dem4_fn).reproject(old_dem)
        refdem = refdem.reproject(old_dem)
        
        # Load stable surfaces mask
        ss_mask_fn = os.path.join(data_dir, site_name, date, 'stable_surfaces', 'stable_surfaces_mask.tif')
        ss_mask = gu.Raster(ss_mask_fn).reproject(old_dem)
        ss_mask = (ss_mask == 1)
        
        # Set up figure
        fig, ax = plt.subplots(3, 2, figsize=(12,14))
        ax = ax.flatten()
        titles = ['Original', 'with refDEM, $\sigma$ = 10 m', 
                  'with refDEM, $\sigma$ = 1 m', 'with refDEM, $\sigma$ = 0.1 m',
                  'with refDEM, $\sigma$ = 1 m, SS only']
        for i, (dem, title) in enumerate(list(zip([old_dem, dem1, dem2, dem3, dem4], titles))):
            # Calculate dDEM
            ddem = dem - refdem
            # Calculate stable surface stats
            ddem_ss = ddem[ss_mask]
            ddem_ss_median, ddem_ss_nmad = np.ma.median(ddem_ss), xdem.spatialstats.nmad(ddem_ss)
            # Plot
            ddem.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax[i], add_cbar=False)
            ax[i].text(602.6e3, 4.8692e6, f"SS median = {np.round(float(ddem_ss_median),2)}\nSS NMAD = {np.round(float(ddem_ss_nmad),2)}")
            ax[i].set_title(title)
            ax[i].set_xticks([])
            ax[i].set_yticks([])
        
        fig.suptitle(f"{site_name}\n{os.path.basename(date)}")
        fig.tight_layout()
        plt.show()
